In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

# Load the sorvetes dataset
print("Loading Data...")
sorvetes = pd.read_csv('sorvetes.csv')

# Separate features and labels
X, y = sorvetes[['Temperatura']].values, sorvetes['Vendas'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print(f"Training a logistic regression model with regularization rate of {reg}")
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print("Accuracy:", acc)

# Calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print("AUC:", str(auc))

# Submit job to Azure ML
from azure.ai.ml import command

# Configure job settings
job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="sorvetes-pythonv2-train",
    experiment_name="sorvetes-training"
)

# Submit job to Azure ML workspace
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
